In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import label_ranking_average_precision_score

In [2]:
#Get true predictions
NUM_FEATURES = 5000
NUM_CLASSES = 3993
y_val = pd.read_csv("../data/expanded/dev_labels.csv", names=range(NUM_CLASSES))

In [3]:
#Tree probabilities
y_rf = pd.read_csv("../public data/saved_rf_probabilities.csv", names=range(NUM_CLASSES),header= 0)

In [7]:
score_rf = label_ranking_average_precision_score(y_val,y_rf)
score_rf

0.5604580391775857

In [18]:
#KNN
y_knn = pd.read_csv("../public data/knn.csv", names=range(NUM_CLASSES))

In [19]:
score_knn = label_ranking_average_precision_score(y_val,y_knn)
score_knn

0.19741457592804107

# Convex combination

In [20]:
#Do a convex combination based upon validation scores
#Note convex combination does not seem to work that well.
total = score_rf + score_knn
rf_w = score_rf / total
knn_w = score_knn / total

In [39]:
#Can use convex combination or set manually.
ensemble = 0.95*y_rf.values + 0.05*y_knn.values

In [40]:
score_ensemble = label_ranking_average_precision_score(y_val,ensemble)

In [41]:
#Small improvement over sole rf
score_ensemble

0.5629898464672906